# Testing Different SN Methods

In [1]:
import numpy as np
import xarray as xr
import os
import matplotlib.pyplot as plt


In [2]:
from matlab_config import matlab_config
matlab_config()

In [3]:
import sys
sys.path.append('../../')
import constants
sys.path.append(constants.MODULE_DIR)
import xarray_class_accessors as xca
import signal_to_noise 
import sn_plotting 
from classes import ExperimentTypes
import open_ds
from utils import pprint_list
from warnings import filterwarnings
filterwarnings('ignore')
import utils
from importlib import reload

In [4]:
ROOT_DIR = os.path.join(constants.LONGRUNMIP_DIR, experiment_params["variable"], 'regrid_retimestamped')
ROOT_DIR

NameError: name 'experiment_params' is not defined

In [6]:
reload(open_ds)

<module 'open_ds' from '/home/563/ab2313/Documents/PhD/modules/open_ds.py'>

In [ ]:
models_to_get = open_ds.get_models_longer_than_length()
utils.pprint_list(models_to_get, num_start_items=7)

In [ ]:
files_to_open_experiment = open_ds.get_file_names_from_from_directory(
    ROOT_DIR, ExperimentTypes.ABRUPT4X, models_to_get)

files_to_open_control = open_ds.get_file_names_from_from_directory(
    ROOT_DIR, ExperimentTypes.CONTROL, models_to_get)

utils.pprint_list(files_to_open_experiment, num_start_items=5)

In [ ]:
control_ds = open_ds.read_and_merge_netcdfs(files_to_open_control, ROOT_DIR, 
                                            mask=experiment_params['mask'])
abrupt4x_ds = open_ds.read_and_merge_netcdfs(files_to_open_experiment, ROOT_DIR,
                                             mask=experiment_params['mask'])
abrupt4x_ds

In [ ]:
abrupt4x_mean,control_ds_mean = signal_to_noise.calculate_global_value(abrupt4x_ds, control_ds, 
                                                          experiment_params["variable"],
                                                constants.HEMISPHERE_LAT[experiment_params['hemisphere']])

In [ ]:
abrupt4x_mean


In [ ]:
da = abrupt4x_mean.cesm104
da

In [ ]:
plt.plot(da.values)

In [ ]:
loess_list = []
for window in (5, 10, 20, 40, 80, 120, 160):
    da_loess_x = da.sn.apply_loess_filter(window=window)
    loess_list.append(da_loess_x.expand_dims("window").assign_coords(window=('window', [window])))

In [ ]:
da_loess = xr.concat(loess_list, dim='window')

In [ ]:
fig = plt.figure(figsize=(15,20))
windows = da_loess.window.values
num_rows = len(windows)

for plot_num, window in enumerate(windows):
    ax = fig.add_subplot(num_rows, 1, plot_num+1)
    ax.plot(da_loess.sel(window=window).values[:500])
    ax.set_title(f'{window=}')

In [ ]:
fig = plt.figure(figsize=(15,8))
plt.plot(da_loess.sel(window=10).values[:500])